In [ ]:
using Revise, Plots, LinearAlgebra, Random

In [ ]:
includet("../../src/SparseSpikes.jl")
using .SparseSpikes

1D

In [ ]:
domain  = [0, 1]

In [ ]:
n_coarse_grid = 21
n_plt_grid = 1001

coarse_grid = grid(domain, n_coarse_grid)
plt_grid = grid(domain, n_plt_grid)

plot_size = (400, 250) .* 2
plt = plot(xlims=extrema(domain), ylims=(-3, 3), size=plot_size, legend=:none)
hline!(plt, [0], linestyle=:solid, color=:black)

In [ ]:
ops = gaussian_operators_1D(0.06, coarse_grid)

## Pre-certificate

In [ ]:
# Easy case
# x0 = [0.2, 0.8]
# a0 = [1.0, -1.0]

# Slightly harder case
# x0 = [0.21, 0.44, 0.63, 0.82]
# a0 = [0.91, -0.78, 0.56, -0.86]

# Close positive spikes
# x0 = [0.47, 0.53]
# a0 = [1.0, 1.0]

# Degenerate case
# x0 = [0.46, 0.54]
# a0 = [1.0, -1.0]

# 9 spikes
x0 = [0.043884, 0.197886, 0.247599, 0.391996, 0.479156, 0.589361, 0.732977, 0.901393, 0.990329]
a0 = [0.779985, 0.543818, 0.623665, -0.780151, 0.699169, 0.472221, 0.434011, -0.235746, -0.096867]

μ0 = DiscreteMeasure(x0, a0)

ηV = build_ηV(μ0, ops)

display(compute_η_max(ηV, coarse_grid))

plt_cert = deepcopy(plt)

plot!(plt_cert, plt_grid, ηV(plt_grid), color=:green)
plot_spikes!(plt_cert, μ0, marker=:square, color=:gold)

In [ ]:
y0 = ops.Φ(μ0...)

function add_noise(y0, δ)
    w = randn(length(y0))
    w = w / norm(w) * δ
    y = y0 + w
    return y
end

# Add noise to the observation y = y0 + w
noise_level = 0.0
Random.seed!(16)
δ = noise_level * norm(y0)
y = add_noise(y0, δ)

τδ = norm(ops.Φ(μ0...) - y) + max(5e-2, δ)

plt_obs = deepcopy(plt)

plot!(plt_obs, plt_grid, ops.adjΦ(y, grid=plt_grid), color=:blue)
plot_spikes!(plt_obs, μ0, marker=:square, color=:gold)

In [ ]:
plt_MDP = deepcopy(plt_obs)

prob_MDP = BLASSO(y, ops, domain, n_coarse_grid)
solve!(prob_MDP, :MDP, options=Dict(:verbose => false, :maxits => 1000, :inner_maxits => 15, :store_reg_path => true, :τδ => τδ, :q => 0.99, :η_tol => 1e-5, :warm_start => true))
display(prob_MDP.λ)
plot_spikes!(plt_MDP, prob_MDP.μ, marker=:circle, color=:green)
plot!(plt_grid, prob_MDP.η(plt_grid))

In [ ]:
λs, μs = values(prob_MDP.reg_path)
x_paths, a_paths = build_reg_paths(λs, μs, C=1e10)
plt_MDP_paths = plot_reg_paths(λs, x_paths, a_paths, size=(800, 250))

In [ ]:
plt_NODE = deepcopy(plt_obs)

prob_NODE = BLASSO(y, ops, domain, n_coarse_grid)
solve!(prob_NODE, :NODE, options=Dict(:verbose => false, :store_reg_path => true, :maxits => 5000, :τδ => τδ, :h => 1e-3, :η_tol => 1e-5, :inner_maxits => 15, :λ₀ => 3))
display(prob_NODE.λ)
plt_NODE = deepcopy(plt_obs)
plot_spikes!(plt_NODE, prob_NODE.μ, color=:green)
plot!(plt_grid, prob_NODE.η(plt_grid))

In [ ]:
λs, μs = values(prob_NODE.reg_path)

λ_step = 1
idxs = 1:λ_step:length(λs)
λs, μs = λs[idxs], μs[idxs]

x_paths, a_paths = build_reg_paths(λs, μs, C=1e100)
plt_NODE_paths = plot_reg_paths(λs, x_paths, a_paths, size=(800, 250))

In [ ]:
plot(plt_MDP_paths, plt_NODE_paths; layout=(1, 2), size=(1000, 700), title=["MDP" "" "NODE" ""])